In [1]:
#from __future__ import print_function

import os 
try:    
    import folium
except ImportError:
    !pip install --user folium
    import folium
    
from folium import plugins
import pandas as pd
import numpy as np

## Data Prep

In [12]:
data=pd.read_csv("/home/dario/NeoMeetup/Visualizations/map/export_european_capitals.csv")
data.head()
data.sort_values("ID_event").head()

,ID_member,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist
157797,218084573,106866582,Paris,48.86,2.34,NaN,NaN,NaN,0.000000
161868,278846290,115197702,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,0.000000
111343,278257631,123953142,London,51.55,-0.17,NaN,NaN,NaN,5.886219
23139,12879421,128105672,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,0.000000
122132,139972652,131523832,London,51.50,-0.19,NaN,NaN,NaN,7.939657


In [13]:
print(len(data))

168077


In [14]:
for line in data.itertuples():
    if data.iloc[line.Index]['dist']==0:
        data['dist'].at[line.Index]=7 #arbitrary for milan
data.head()   

,ID_member,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist
0,266338314,260067860,Bucharest,44.44,26.10,IBM Romania,44.444588,26.060389,7.000000
1,266338314,260081179,Bucharest,44.44,26.10,Veridium,44.450085,26.083150,7.000000
2,37748782,260486574,Budapest,47.51,19.08,Kapt\u00e1r Irod\u00e1k Kft.,47.501040,19.055702,9802.479459
3,197132338,260500250,London,51.52,-0.10,Tate Britain,51.492425,-0.125338,76.889705
4,71303262,260172898,Stockholm,59.33,18.07,Foo Caf\u00e9,59.303745,18.084581,7.000000


In [15]:
data.sort_values(["ID_event","dist"], inplace=True)
data

,ID_member,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist
157797,218084573,106866582,Paris,48.86,2.34,NaN,NaN,NaN,7.000000
161868,278846290,115197702,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000
111343,278257631,123953142,London,51.55,-0.17,NaN,NaN,NaN,5.886219
23139,12879421,128105672,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000
122132,139972652,131523832,London,51.50,-0.19,NaN,NaN,NaN,7.939657
93021,55734112,150138502,Paris,48.86,2.34,NaN,NaN,NaN,874.785710
44075,157861502,162145372,Stockholm,59.33,18.07,NaN,NaN,NaN,7.000000
4761,60378,16537690,Paris,48.86,2.34,NaN,NaN,NaN,7.000000
98123,91455112,168469672,Berlin,52.52,13.38,NaN,NaN,NaN,7.000000
87495,277957260,175163972,London,51.52,-0.10,NaN,NaN,NaN,81.525620


In [16]:
ddrop=data.drop_duplicates(subset="ID_event" , keep="last")
ddrop=ddrop.drop("ID_member", axis=1)
#ddrop.sort_values("ID_event")
ddrop=ddrop.reset_index()
ddrop.head()

,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist
0,157797,106866582,Paris,48.86,2.34,NaN,NaN,NaN,7.000000
1,161868,115197702,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000
2,111343,123953142,London,51.55,-0.17,NaN,NaN,NaN,5.886219
3,23139,128105672,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000
4,122132,131523832,London,51.50,-0.19,NaN,NaN,NaN,7.939657


In [17]:
d=data.groupby("ID_event")['dist'].quantile(0.75).reset_index()
print(d.head())
d.rename(columns={'dist':'maxdist'}, inplace=True)
ddrop=ddrop.merge(d, on="ID_event")
ddrop.head()

    ID_event      dist
0  106866582  7.000000
1  115197702  7.000000
2  123953142  5.886219
3  128105672  7.000000
4  131523832  7.939657


,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist,maxdist
0,157797,106866582,Paris,48.86,2.34,NaN,NaN,NaN,7.000000,7.000000
1,161868,115197702,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000,7.000000
2,111343,123953142,London,51.55,-0.17,NaN,NaN,NaN,5.886219,5.886219
3,23139,128105672,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000,7.000000
4,122132,131523832,London,51.50,-0.19,NaN,NaN,NaN,7.939657,7.939657


In [18]:
d1=data.groupby("ID_event")['ID_member'].count().reset_index()
print(d1.head())

    ID_event  ID_member
0  106866582          1
1  115197702          1
2  123953142          1
3  128105672          1
4  131523832          1


In [19]:
d1.rename(columns={'ID_member':'count_part'}, inplace=True)
ddrop=ddrop.merge(d1, on="ID_event")
ddrop.head()

,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist,maxdist,count_part
0,157797,106866582,Paris,48.86,2.34,NaN,NaN,NaN,7.000000,7.000000,1
1,161868,115197702,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000,7.000000,1
2,111343,123953142,London,51.55,-0.17,NaN,NaN,NaN,5.886219,5.886219,1
3,23139,128105672,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000,7.000000,1
4,122132,131523832,London,51.50,-0.19,NaN,NaN,NaN,7.939657,7.939657,1


## Let's build a map

In [20]:
#legend template
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: fixed; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Bacino d'utenza</li>
    <!-- questo è un commento <li><span style='background:orange;opacity:0.7;'></span>Medium</li>
    <li><span style='background:green;opacity:0.7;'></span>Small</li> -->

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""



macro = MacroElement()
macro._template = Template(template)

html = """
    <h3> Event ID  : </h3><p>{}</p>
    <h3> Event Name: </h3><p></p>
    <h3> Location: </h3><p>{}</p>
    """

legend_html = """
     <div style=”position: fixed; 
     bottom: 50px; left: 50px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     “>&nbsp; Cool Legend <br>
     &nbsp; East &nbsp; <i class=”fa fa-map-marker fa-2x”
                  style=”color:green”></i><br>
     &nbsp; West &nbsp; <i class=”fa fa-map-marker fa-2x”
                  style=”color:red”></i>
      </div>
     """
#m.get_root().add_child(macro)

#m

In [21]:
import branca #brrr
#m1 = folium.Map(location=[20,0], tiles="Mapbox Bright", zoom_start=2)
from folium.plugins import MarkerCluster
from folium.features import DivIcon

m = folium.Map(location=[ddrop['v_lat'].mean(),ddrop['v_lon'].mean()], zoom_start=5)
mc=MarkerCluster()

for line in ddrop.itertuples():
    #try:
        if str(line.v_lat) != "nan" : #where exists use venue lat/lon
            lat=line.v_lat
        else: #elsewhere use group lat
            lat=line.g_lat
            #print("g_lat: ",lat)
        if str(line.v_lon) != "nan": #where exists use venue lat/lon
            lon=line.v_lon
        else: #elsewhere use group lat
            lon=line.g_lon

        f = branca.element.Figure()
        m1=folium.Map([float(lat), float(lon)], zoom_start=10) #eventualmente aggiungere il livello di zoom in base al raggio può essere fyco

        iframe = branca.element.IFrame(
            html=html.format(ddrop['ID_event'].at[line.Index], ddrop['venue_name'].at[line.Index]), 
            width=400, height=240)
        popup1 = folium.Popup(iframe, show=True)


        m1.add_child(folium.Circle(
          location=[float(lat), float(lon)],
          popup=popup1,
          radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
          #radius=1,
          color='crimson',
          fill=False,
          #fill_color='crimson',

        ))
        #print (lat,"   ", lon)
        #trying to add some text
        #m1.add_child(folium.Marker(
            #location=[45.5,8.5],#[lat, lon],
            #icon=DivIcon(icon_size=(150,36), icon_anchor=(0,0),html='<div style="font-size: 24pt">Bacino d\'utenza</div>')
            #))

        #macro1 = MacroElement()
        #macro1._template = Template(template)
        #m1.get_root().add_child(macro1)

        f.html.add_child(branca.element.Element("<h1>Bacino d'utenza</h1>"))
        #f.html.add_child(branca.element.Element(legend_html))

        #fig = folium.element.Figure()
        #fig.html.add_child(folium.element.Element("<h1>This is a title</h1>"))

        #m1.title = 'My fancy title here'
        #f.add_child(m1)
        m1.add_to(f)

        #iframe = branca.element.IFrame(html=html.format(ddrop['event_id'].at[line.Index], ddrop['city'].at[line.Index]), width=500, height=300)
        #popup = folium.Popup(iframe)
        iframe = branca.element.IFrame(width=1000, height=600)
        f.add_to(iframe)

        # Let's put the IFrame in a Popup
        popup = folium.Popup(iframe, max_width=2650)
        mc.add_child(
            folium.Marker(
            location = [float(lat), float(lon)], 
            popup=popup,
            ))
        #if line.Index is 10:
        #    break
    #except Exception as e:
    #    print(e)
m.add_child(mc)
plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)


macro = MacroElement()

macro._template = Template(template)

m.get_root().add_child(macro)
#map.get_root().add_child(macro1)

folium.LayerControl().add_to(m)

#display(m)
    
#Save it as html
m.save('/home/dario/NeoMeetup/Visualizations/map/export_europe_capitals.html')

In [24]:
ddrop.head()

,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist,maxdist,count_part
0,157797,106866582,Paris,48.86,2.34,NaN,NaN,NaN,7.000000,7.000000,1
1,161868,115197702,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000,7.000000,1
2,111343,123953142,London,51.55,-0.17,NaN,NaN,NaN,5.886219,5.886219,1
3,23139,128105672,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000,7.000000,1
4,122132,131523832,London,51.50,-0.19,NaN,NaN,NaN,7.939657,7.939657,1


In [40]:
lat_df=ddrop[['v_lat', 'v_lon']]
print(len(ddrop))
str(lat_df.v_lat[0]) == "nan"

for line in lat_df.itertuples():
    if str(line.v_lat) == "nan":
        lat_df['v_lat'].at[line.Index]= ddrop['g_lat'].at[line.Index]
    if str(line.v_lon) == "nan":
        lat_df['v_lon'].at[line.Index]= ddrop['g_lon'].at[line.Index]
len(lat_df)

lat_df.isna().any()

lat_df=lat_df.astype(float)

from folium.plugins import HeatMap


map_hooray = folium.Map(location=[ddrop['v_lat'].mean(),ddrop['v_lon'].mean()], zoom_start=5)


# List comprehension to make out list of lists
heat_data = [[row['v_lat'],row['v_lon']] for index, row in lat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_hooray)

# Display the map
map_hooray.save("/home/dario/NeoMeetup/Visualizations/map/heat_try.html")





v_lat    False
v_lon    False
dtype: bool